In [1]:
import torch
import torchvision

NUM_TIMESTEPS = 128

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, NUM_TIMESTEPS)

device = torch.device('cuda')

model.load_state_dict(torch.load('circuit_model.pth'))

model = model.to(device).eval().float()

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('circuit_model_trt.pth'))

In [2]:
import torch2trt
import roch

data = torch.zeros((1, 3, 224, 224)).cuda()

model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True)

ModuleNotFoundError: No module named 'roch'

In [3]:
torch.save(model_trt.state_dict(), 'circuit_model_trt.pth')

In [3]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_width=1280, capture_height=720, capture_fps=60)

camera.running = True

In [4]:
import ipywidgets

image_widget = ipywidgets.Image(format='jpeg')

image_widget

Image(value=b'', format='jpeg')

In [5]:
NUM_TIMESTEPS = 128

In [64]:
import cv2
import os
from utils import preprocess


def execute(change):
    image = change['new']
    data = preprocess(image)
    
    # set steering
    xy = apex_model_trt(data)[0].cpu()
    x = float(xy[0])
    car.steering = x * steering_gain.value + steering_offset.value
    
    # set speed
    output = model_trt(data)[0].detach().cpu() - 0.00
    speed = float(torch.exp(-torch.sum(torch.abs(output[0:64])) * 0.1))
#     speed = float(torch.mean(torch.exp(-torch.abs(output) * 1.0)))
#     scale = torch.exp(-torch.linspace(0, 1.0, NUM_TIMESTEPS))

#     speed = float(torch.exp(-10.0 * torch.sum(torch.abs(output) * scale) / scale.sum()))
    speed = throttle_gain.value * speed + throttle_offset.value
    car.throttle = speed
    
#     # visualize
#     x_i = int(112 + float(xy[0]) * 112)
    y_i = int(112 + float(xy[1]) * 112)
    cv2.circle(image, (x_i, y_i), 5, (255, 0, 0), 3)
    
    for i in range(NUM_TIMESTEPS):
        x = int(float(output[i]) * 112 + 112)
        y = int(224 - 224 * i / NUM_TIMESTEPS)
        image = cv2.circle(image, (x, y), 2, (0, 255, 0), 2)
    cv2.line(image, (112, 0), (112, 224), (255, 0, 0), 2)
    
    
    cv2.putText(image, '%f' % speed, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
    
    image_widget.value = bgr8_to_jpeg(image)
    

In [45]:
execute({'new': camera.value})

In [65]:
camera.observe(execute, names='value')

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jetcam-0.0.0-py3.6.egg/jetcam/camera.py", line 34, in _capture_frames
    self.value = self._read()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 584, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 573, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1205, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1210, in notify_change
    return self._notify_observers(change)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", l

In [63]:
camera.unobserve_all()

In [6]:
from IPython.display import display
import ipywidgets

steering_gain = ipywidgets.FloatSlider(min=0.0, max=3.0, value=1.5, step=0.001, description='steering gain')
steering_offset = ipywidgets.FloatSlider(min=-0.5, max=0.5, value=0.0, step=0.001, description='steering offset')
throttle_gain = ipywidgets.FloatSlider(min=0.0, max=2.0, value=0.0, step=0.001, description='throttle gain')
throttle_offset = ipywidgets.FloatSlider(min=-0.5, max=0.5, value=0.0, step=0.001, description='throttle offset')

display(
    steering_gain,
    steering_offset,
    throttle_gain,
    throttle_offset
)

FloatSlider(value=1.5, description='steering gain', max=3.0, step=0.001)

FloatSlider(value=0.0, description='steering offset', max=0.5, min=-0.5, step=0.001)

FloatSlider(value=0.0, description='throttle gain', max=2.0, step=0.001)

FloatSlider(value=0.0, description='throttle offset', max=0.5, min=-0.5, step=0.001)

In [7]:
import torch
from torch2trt import TRTModule

apex_model_trt = TRTModule()
apex_model_trt.load_state_dict(torch.load('apex_model_trt.pth'))

In [8]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar(
    steering_gain=0.5,
    steering_offset=0.0,
    throttle_gain=-0.5,
    throttle_offset=0.0
)

In [9]:
car.throttle_gain = -1.0